In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [165]:
# These are used to adjust the visibility of data while working on them
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows', None)

In [2]:
# first the original data is read 
df = pd.read_csv('/Users/user/Downloads/NYC_Permitted_Event_Information_-_Historical (2).csv')
df.head()

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
0,368421.0,Big Apple Circus,11/18/2017 07:00:00 PM,11/18/2017 08:00:00 PM,Parks Department,Special Event,Manhattan,"Damrosch Park: Damrosch Park ,Damrosch Park: T...",NaN,NaN,"7,","20,"
1,330050.0,Mt. Eden Farmer's Market,11/16/2017 08:00:00 AM,11/16/2017 04:00:00 PM,Parks Department,Special Event,Bronx,Mount Eden Malls: Mount Eden Malls,NaN,NaN,"4,","44,"
2,314111.0,Columbia Greenmarket Thursday,11/21/2017 08:00:00 AM,11/21/2017 05:00:00 PM,Street Activity Permit Office,Farmers Market,Manhattan,BROADWAY between WEST 114 STREET and WEST 1...,East,Sidewalk and Curb Lane Closure,"9,","26,"
3,369850.0,Lawn Maintenance,11/23/2017 12:00:00 AM,11/23/2017 11:58:00 PM,Parks Department,Construction,Manhattan,Madison Square Park: Center Lawn,NaN,NaN,"5,","13,"
4,335783.0,"October, November December model aircraft flying",11/22/2017 09:00:00 AM,11/22/2017 08:00:00 PM,Parks Department,Special Event,Staten Island,LaTourette Park & Golf Course: Model Airplane ...,NaN,NaN,"2,","122,"


In [4]:
df.shape

(16936390, 12)

The dataset is huge with around 17 millions rows of data. My computer was having hard time even just read and open the folder in the notebook. Therefore initial cleaning is very important to eliminate as much irrelevant rows as possible to facilitate the later stage of work.

# Basic Data Cleaning

In [5]:
#Drop all the entries will a NaN value in any of the features, as they are all important for later machine learning stage.
df = df.dropna(subset=['Event ID', 'Event Name','Start Date/Time','End Date/Time','Event Borough','Event Location'])
df.shape

(16936355, 12)

In [6]:
#Select only the events that happen in Manhattan
selected_tuples = df[df['Event Borough'].str.contains ("Manhattan")]
selected_tuples.shape

(6359537, 12)

In [8]:
# check the duplicates entries
selected_tuples[selected_tuples.duplicated()]

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
1973,368421.0,Big Apple Circus,11/18/2017 07:00:00 PM,11/18/2017 08:00:00 PM,Parks Department,Special Event,Manhattan,"Damrosch Park: Damrosch Park ,Damrosch Park: T...",NaN,NaN,"7,","20,"
1975,314111.0,Columbia Greenmarket Thursday,11/21/2017 08:00:00 AM,11/21/2017 05:00:00 PM,Street Activity Permit Office,Farmers Market,Manhattan,BROADWAY between WEST 114 STREET and WEST 1...,East,Sidewalk and Curb Lane Closure,"9,","26,"
1976,369850.0,Lawn Maintenance,11/23/2017 12:00:00 AM,11/23/2017 11:58:00 PM,Parks Department,Construction,Manhattan,Madison Square Park: Center Lawn,NaN,NaN,"5,","13,"
1978,369858.0,Lawn Maintenance,12/06/2017 12:00:00 AM,12/06/2017 11:58:00 PM,Parks Department,Construction,Manhattan,Madison Square Park: Dogwood / 25th St Lawn No...,NaN,NaN,"5,","13,"
1979,363387.0,Crab Apple Grove LAWN CLOSURE,11/18/2017 12:00:00 AM,11/18/2017 11:59:00 PM,Parks Department,Construction,Manhattan,Riverside Park: Crab Apple Grove-RSP,NaN,NaN,"7,","24,"
...,...,...,...,...,...,...,...,...,...,...,...,...
16935901,687023.0,Celebration,12/01/2023 11:00:00 AM,12/01/2023 12:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Ladies' Pavilion,NaN,NaN,"64,","22,"
16936013,687316.0,MUTS,08/18/2023 08:00:00 PM,08/18/2023 10:00:00 PM,Parks Department,Special Event,Manhattan,Luther Gulick Playground: Handball-01,NaN,NaN,"03,","7,"
16936184,679693.0,Celebration,06/24/2023 03:00:00 PM,06/24/2023 05:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Cop Cot,NaN,NaN,"64,","22,"
16936305,680062.0,Celebration,06/25/2023 03:00:00 PM,06/25/2023 05:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Cop Cot,NaN,NaN,"64,","22,"


In [9]:
#drop all the duplicate entries
selected_tuples = selected_tuples.drop_duplicates()

In [10]:
selected_tuples.shape

(461346, 12)

In [25]:
#Sort the result dataset alphabatically by their event name and drop the index
selected_tuples = selected_tuples.sort_values('Event Name')
selected_tuples.reset_index(drop=True, inplace=True)
selected_tuples = selected_tuples.drop('index', axis=1)
selected_tuples 

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
0,453104.0,Warbug Pincus,12/13/2018 06:00:00 AM,12/13/2018 11:59:00 PM,Street Activity Permit Office,Production Event,Manhattan,WEST 46 STREET between 11 AVENUE and 12 AVENUE,North,Curb Lane Only,"4,","18,"
1,509900.0,Abbott Dash to the Finish Line 5k,11/02/2019 02:00:00 AM,11/02/2019 01:00:00 PM,Street Activity Permit Office,Street Event,Manhattan,1 AVENUE between EAST 47 STREET and EAST ...,Both,Sidewalk and Curb Lane Closure,"5, 6, 64,","14, 17, 18, 22,"
2,471438.0,Artist Scott Campbell,04/01/2019 12:00:00 PM,04/03/2019 06:00:00 PM,Street Activity Permit Office,Production Event,Manhattan,MADISON AVENUE between EAST 60 STREET and E...,East,Partial Sidewalk Closure,"8,","19,"
3,561754.0,Baseball - 12 and Under (Little League),05/27/2021 04:00:00 PM,05/27/2021 08:00:00 PM,Parks Department,Sport - Youth,Manhattan,Randall's Island Park: Sunken Meadow Fields-So...,NaN,NaN,"11,","25,"
4,561754.0,Baseball - 12 and Under (Little League),05/28/2021 04:00:00 PM,05/28/2021 08:00:00 PM,Parks Department,Sport - Youth,Manhattan,Randall's Island Park: Sunken Meadow Fields-So...,NaN,NaN,"11,","25,"
...,...,...,...,...,...,...,...,...,...,...,...,...
461341,523369.0,youth baseball,10/14/2020 03:00:00 PM,10/14/2020 06:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: West 96th Street Entrance,NaN,NaN,"64,","22,"
461342,523369.0,youth baseball,10/15/2020 03:00:00 PM,10/15/2020 06:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: West 96th Street Entrance,NaN,NaN,"64,","22,"
461343,523369.0,youth baseball,10/02/2020 03:00:00 PM,10/02/2020 06:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: West 96th Street Entrance,NaN,NaN,"64,","22,"
461344,523369.0,youth baseball,10/07/2020 03:00:00 PM,10/07/2020 06:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: West 96th Street Entrance,NaN,NaN,"64,","22,"


In [69]:
#Save the cleaned dataset to a separate documents
selected_tuples.to_csv('events_cleaned.csv', index=False)

The size of the dataset is brought down to less than 500,000 entries which is significantly smaller than what it was. Rows that does not fulfill any of the conditions aboved were dropped directly, primarily due to the gigantic size of the dataset where keeping the dataset as simple and small as possible is our main target so as my computer can handle the operations after.

# Event Selections

In [42]:
#Check the top 30 events in the dataset
top_events = selected_tuples['Event Name'].value_counts().head(30)
top_events

Soccer - Non Regulation                     32581
Miscellaneous                               31807
 Soccer - Non Regulation                    27999
 Soccer -Regulation                         21017
Soccer -Regulation                          18607
Celebration                                 14687
 Softball - Adults                          13170
 Baseball - 12 and Under (Little League)    13096
Baseball - 12 and Under (Little League)     12105
Softball - Adults                           10929
Party                                        8184
Athletic Event Non Sports                    7662
 Baseball - 13 and Older (Little League)     6527
Baseball - 13 and Older (Little League)      5615
Softball (Little league)                     5615
Picnic                                       5426
Lawn Maintenance                             5183
 Softball (Little league)                    4924
Basketball                                   4070
Miscellaneous                                3000


In [40]:
selected_tuples["Event Name"].nunique()

37303

Initially, the plan was to look for the events with top 10 highest frequency in the data. As we can see from the result above, however, the data are full of minor events which has little significance in further examination.

As a result, a different approach was taken by carrying out online research on the popular annual events took place in NYC and then extract the corresponding data out from this dataset. A detailed dev log for the work is included in the Notion under "Data Team". The link is here: https://www.notion.so/Curated-list-of-events-bf8df850b5c14cd99545cdcd12d184fa

# Curated events data exploration

## Macy's Thanksgiving Day Parade

In [96]:
event_1 = selected_tuples[selected_tuples['Event Name'].str.contains ("Macy's Thanksgiving Day Parade")]
event_1.head(50)

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
106713,130574.0,2013 Macy's Thanksgiving Day Parade - NBC Broadcast,11/28/2013 09:00:00 AM,11/28/2013 12:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,"BROADWAY between WEST 34 STREET and WEST 35 STREET, BROADWAY between WEST 33 STREET and WEST 34 STREET, WEST 34 STREET between BROADWAY and 7 AVENUE",Both,Pedestrian Island,"5,","14,"
106824,178873.0,2014 Macy's Thanksgiving Day Parade - NBC Broadcast,11/27/2014 09:00:00 AM,11/27/2014 12:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,"34/35 Broadway Pedestrian Plaza (Herald Square): 34/35 Broadway Pedestrian Plaza (Herald Square) BROADWAY between W 34TH STREET and W. 35TH STREET, BROADWAY between WEST 33 STREET and WEST 34 STREET, WEST 34 STREET between BROADWAY and 7 AVENUE",NaN,Pedestrian Plaza,"5,","14,"
108337,448280.0,2018 Verizon 360 Broadcast Macy's Thanksgiving Day Parade,11/23/2018 12:00:00 AM,11/23/2018 12:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: 6th Avenue Entrance,NaN,NaN,"64,","22,"
108591,513526.0,2019 Verizon 360 NBC Macy's Thanksgiving Day Parade,11/28/2019 06:00:00 AM,11/28/2019 01:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: 6th Avenue Entrance,NaN,NaN,"64,","22,"
112280,11288.0,83rd Annual Macy's Thanksgiving Day Parade,11/26/2009 10:00:00 AM,11/26/2009 02:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,"WEST 77 STREET between CENTRAL PARK WEST and COLUMBUS AVENUE, CENTRAL PARK WEST between WEST 77 STREET and WEST 86 STREET, COLUMBUS AVENUE between WEST 77 STREET and WEST 81 STREET, WEST 34 STREET between BROADWAY and 7 AVENUE, BROADWAY between WEST 34 STREET and WEST 35 STREET, WEST 34 STREET between BROADWAY and 8 AVENUE, BROADWAY between WEST 34 STREET and WEST 36 STREET, WEST 35 STREET between BROADWAY and 6 AVENUE, WEST 36 STREET between 5 AVENUE and 6 AVENUE, WEST 35 STREET between 5 AVENUE and 6 AVENUE",Both,Full Street Closure,"4, 5, 7,","14, 20,"
112285,75680.0,86th Annual Macy's Thanksgiving Day Parade,11/22/2012 09:00:00 AM,11/23/2012 12:30:00 AM,Street Activity Permit Office,Special Event,Manhattan,"COLUMBUS AVENUE between WEST 77 STREET and WEST 81 STREET, WEST 35 STREET between BROADWAY and 7 AVENUE, WEST 34 STREET between BROADWAY and 7 AVENUE, BROADWAY between WEST 34 STREET and WEST 35 STREET, WEST 36 STREET between BROADWAY and 6 AVENUE, WEST 36 STREET between 5 AVENUE and 6 AVENUE, WEST 35 STREET between 5 AVENUE and 6 AVENUE, BROADWAY between WEST 35 STREET and WEST 36 STREET, WEST 59 STREET between 6 AVENUE and 5 AVENUE, WEST 34 STREET between 5 AVENUE and 6 AVENUE, WEST 76 STREET between COLUMBUS AVENUE and AMSTERDAM AVENUE, WEST 71 STREET between COLUMBUS AVENUE and AMSTERDAM AVENUE, WEST 36 STREET between 8 AVENUE and 7 AVENUE",East,Curb Lane Only,"5, 64, 7,","14, 18, 20, 22,"
112286,68213.0,86th Annual Macy's Thanksgiving Day Parade - Grandstand Seating,11/12/2012 12:01:00 AM,11/12/2012 11:59:00 PM,Parks Department,Special Event,Manhattan,"Central Park: Central Park West Sidewalks - 65th St to 72nd St ,Central Park: Central Park South Sidewalks - 6th Ave to 7th Ave",NaN,NaN,"64,","22,"
112287,68213.0,86th Annual Macy's Thanksgiving Day Parade - Grandstand Seating,11/10/2012 12:01:00 AM,11/10/2012 11:59:00 PM,Parks Department,Special Event,Manhattan,"Central Park: Central Park West Sidewalks - 65th St to 72nd St ,Central Park: Central Park South Sidewalks - 6th Ave to 7th Ave",NaN,NaN,"64,","22,"
112288,68213.0,86th Annual Macy's Thanksgiving Day Parade - Grandstand Seating,11/16/2012 12:01:00 AM,11/16/2012 11:59:00 PM,Parks Department,Special Event,Manhattan,"Central Park: Central Park West Sidewalks - 65th St to 72nd St ,Central Park: Central Park South Sidewalks - 6th Ave to 7th Ave",NaN,NaN,"64,","22,"
112289,68213.0,86th Annual Macy's Thanksgiving Day Parade - Grandstand 

In [101]:
event_1 = event_1.drop(event_1[(event_1['Event Name'].str.contains ("Grandstand Seating"))].index)
event_1 = event_1.drop(event_1[(event_1['Event Name'].str.contains ("Preparation"))].index)                     
event_1 = event_1.drop(event_1[(event_1['Event Name'].str.contains ("93rd Annual Macy's Thanksgiving Day Parade")) & ~(event_1['Start Date/Time'].str.contains ("28"))].index)
event_1 = event_1.drop(event_1[(event_1['Event Name'].str.contains ("96th Annual Macy's Thanksgiving Day Parade")) & ~(event_1['Start Date/Time'].str.contains ("24"))].index)
event_1 = event_1.drop(event_1[(event_1['Event Name'].str.contains ("87th Annual Macy's Thanksgiving Day Parade")) & ~(event_1['Start Date/Time'].str.contains ("28"))].index)
event_1 = event_1.drop(event_1[(event_1['Event Name'].str.contains ("92nd Annual Macy's Thanksgiving Day Parade")) & ~(event_1['Start Date/Time'].str.contains ("22"))].index)
event_1 = event_1[event_1['Event Name'].str.contains ("Annual Macy's Thanksgiving Day Parade")& ~(selected_tuples['Event Name'].str.contains ("Preparation"))]

C:\Users\user\AppData\Local\Temp\ipykernel_25304\3359845809.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  event_1 = event_1[event_1['Event Name'].str.contains ("Annual Macy's Thanksgiving Day Parade")& ~(selected_tuples['Event Name'].str.contains ("Preparation"))]


In [102]:
event_1

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
112280,11288.0,83rd Annual Macy's Thanksgiving Day Parade,11/26/2009 10:00:00 AM,11/26/2009 02:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,"WEST 77 STREET between CENTRAL PARK WEST and COLUMBUS AVENUE, CENTRAL PARK WEST between WEST 77 STREET and WEST 86 STREET, COLUMBUS AVENUE between WEST 77 STREET and WEST 81 STREET, WEST 34 STREET between BROADWAY and 7 AVENUE, BROADWAY between WEST 34 STREET and WEST 35 STREET, WEST 34 STREET between BROADWAY and 8 AVENUE, BROADWAY between WEST 34 STREET and WEST 36 STREET, WEST 35 STREET between BROADWAY and 6 AVENUE, WEST 36 STREET between 5 AVENUE and 6 AVENUE, WEST 35 STREET between 5 AVENUE and 6 AVENUE",Both,Full Street Closure,"4, 5, 7,","14, 20,"
112285,75680.0,86th Annual Macy's Thanksgiving Day Parade,11/22/2012 09:00:00 AM,11/23/2012 12:30:00 AM,Street Activity Permit Office,Special Event,Manhattan,"COLUMBUS AVENUE between WEST 77 STREET and WEST 81 STREET, WEST 35 STREET between BROADWAY and 7 AVENUE, WEST 34 STREET between BROADWAY and 7 AVENUE, BROADWAY between WEST 34 STREET and WEST 35 STREET, WEST 36 STREET between BROADWAY and 6 AVENUE, WEST 36 STREET between 5 AVENUE and 6 AVENUE, WEST 35 STREET between 5 AVENUE and 6 AVENUE, BROADWAY between WEST 35 STREET and WEST 36 STREET, WEST 59 STREET between 6 AVENUE and 5 AVENUE, WEST 34 STREET between 5 AVENUE and 6 AVENUE, WEST 76 STREET between COLUMBUS AVENUE and AMSTERDAM AVENUE, WEST 71 STREET between COLUMBUS AVENUE and AMSTERDAM AVENUE, WEST 36 STREET between 8 AVENUE and 7 AVENUE",East,Curb Lane Only,"5, 64, 7,","14, 18, 20, 22,"
112309,125155.0,87th Annual Macy's Thanksgiving Day Parade,11/28/2013 09:00:00 AM,11/28/2013 11:00:00 AM,Parks Department,Special Event,Manhattan,"Central Park: Central Park West Sidewalks - 65th St to 72nd St ,Central Park: Central Park South Sidewalks - 7 Ave to Merchant's Gate",NaN,NaN,"64,","22,"
112311,126522.0,87th Annual Macy's Thanksgiving Day Parade,11/28/2013 09:00:00 AM,11/28/2013 12:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,"COLUMBUS AVENUE between WEST 77 STREET and WEST 81 STREET, WEST 34 STREET between BROADWAY and 7 AVENUE, 34/35 Broadway Pedestrian Plaza (Herald Square): 34/35 Broadway Pedestrian Plaza (Herald Square) BROADWAY between W 34TH STREET and W. 35TH STREET, WEST 36 STREET between 5 AVENUE and 6 AVENUE, WEST 35 STREET between 5 AVENUE and 6 AVENUE, BROADWAY between WEST 35 STREET and WEST 36 STREET, WEST 59 STREET between 5 AVENUE and 6 AVENUE, WEST 34 STREET between 5 AVENUE and 6 AVENUE, WEST 47 STREET between 6 AVENUE and 7 AVENUE",East,Curb Lane Only,"5, 64, 7,","14, 18, 20, 22,"
112336,186645.0,88th Annual Macy's Thanksgiving Day Parade,11/27/2014 09:00:00 AM,11/27/2014 12:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,"COLUMBUS AVENUE between WEST 77 STREET and WEST 81 STREET, WEST 34 STREET between BROADWAY and 7 AVENUE, BROADWAY between WEST 34 STREET and WEST 35 STREET, WEST 35 STREET between 5 AVENUE and 6 AVENUE, BROADWAY between WEST 35 STREET and WEST 36 STREET, WEST 59 STREET between 5 AVENUE and 6 AVENUE, WEST 34 STREET between 5 AVENUE and 6 AVENUE, WEST 47 STREET between 6 AVENUE and 7 AVENUE",Full,Full Street Closure,"5, 64, 7,","14, 18, 20, 22,"
112592,437785.0,92nd Annual Macy's Thanksgiving Day Parade,11/22/2018 12:01:00 AM,11/22/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,"Central Park: Central Park West Sidewalks - 72nd St to 81st St ,Central Park: Central Park West Sidewalks - 65th St to 72nd St ,Central Park: Central Park West Sidewalks - Merchant's Gate to 65 St ,Central Park: Merchants' Gate ,Central Park: Central Park South Sidewalks - 7 Ave to Merchant's Gate ,Central Park: Central Park South Sidewalks - 6th Ave to 7th Ave",NaN,NaN,"64,","22,"
112768,506638.0,93rd Annual Macy's Thanksgiving Day Para

## Halloween Parade

In [59]:
event_2 = selected_tuples[selected_tuples['Event Name'].str.contains ("Halloween Parade and Party")]
event_2.head(50)

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
223614,379815.0,Halloween Parade and Party,10/29/2017 12:50:00 PM,10/29/2017 01:20:00 PM,Police Department,Parade,Manhattan,"GREENWICH STREET between NORTH MOORE STREET and BEACH STREET, GREENWICH STREET between NORTH MOORE STREET and DUANE STREET, GREENWICH STREET between DUANE STREET and READE STREET",Full,Full Street Closure,"1,","1,"
223615,210271.0,Halloween Parade and Party,10/25/2015 01:00:00 PM,10/25/2015 01:30:00 PM,Police Department,Parade,Manhattan,"GREENWICH STREET between NORTH MOORE STREET and BEACH STREET, GREENWICH STREET between NORTH MOORE STREET and DUANE STREET, GREENWICH STREET between DUANE STREET and CHAMBERS STREET",Full,Full Street Closure,"1,","1,"
223616,175929.0,Halloween Parade and Party,10/26/2014 01:00:00 PM,10/26/2014 03:00:00 PM,Parks Department,Special Event,Manhattan,"Washington Market Park: Center Lawn ,Washington Market Park: Gazebo",NaN,NaN,"1,","1,"
223617,446834.0,Halloween Parade and Party,10/28/2018 12:50:00 PM,10/28/2018 01:20:00 PM,Police Department,Parade,Manhattan,"GREENWICH STREET between NORTH MOORE STREET and BEACH STREET, GREENWICH STREET between NORTH MOORE STREET and DUANE STREET, GREENWICH STREET between DUANE STREET and READE STREET",Full,Full Street Closure,"1,","1,"
223618,207005.0,Halloween Parade and Party,10/25/2015 01:00:00 PM,10/25/2015 03:00:00 PM,Parks Department,Special Event,Manhattan,"Washington Market Park: Washington Market Park ,Washington Market Park: Center Lawn ,Washington Market Park: Gazebo",NaN,NaN,"1,","1,"
223619,315443.0,Halloween Parade and Party,10/30/2016 12:50:00 PM,10/30/2016 01:20:00 PM,Police Department,Parade,Manhattan,"GREENWICH STREET between NORTH MOORE STREET and BEACH STREET, GREENWICH STREET between NORTH MOORE STREET and DUANE STREET, GREENWICH STREET between DUANE STREET and READE STREET",Full,Full Street Closure,"1,","1,"
223620,301217.0,Halloween Parade and Party,10/30/2016 01:00:00 PM,10/30/2016 03:00:00 PM,Parks Department,Special Event,Manhattan,"Washington Market Park: Washington Market Park ,Washington Market Park: Center Lawn ,Washington Market Park: Gazebo",NaN,NaN,"1,","1,"
223621,179932.0,Halloween Parade and Party,10/26/2014 01:00:00 PM,10/26/2014 01:30:00 PM,Police Department,Parade,Manhattan,"GREENWICH STREET between NORTH MOORE STREET and BEACH STREET, GREENWICH STREET between NORTH MOORE STREET and DUANE STREET, Washington Market Park: Sidewalk",Full,Full Street Closure,"1,","1,"
223622,406904.0,Halloween Parade and Party,10/28/2018 01:00:00 PM,10/28/2018 03:00:00 PM,Parks Department,Special Event,Manhattan,"Washington Market Park: Washington Market Park ,Washington Market Park: Center Lawn ,Washington Market Park: Gazebo",NaN,NaN,"1,","1,"
223623,511310.0,Halloween Parade and Party,10/27/2019 12:50:00 PM,10/27/2019 01:20:00 PM,Police Department,Parade,Manhattan,"GREENWICH STREET between NORTH MOORE STREET and BEACH STREET, GREENWICH STREET between NORTH MOORE STREET and DUANE STREET, GREENWICH STREET between DUANE STREET and READE STREET",Full,Full Street Closure,"1,","1,"


# St.Patrick's Day Parade

In [115]:
event_3 = selected_tuples[selected_tuples['Event Name'].str.contains ("NYC Saint ")]
event_3 

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
303839,462971.0,NYC Saint PAtrick's Day Parade,03/16/2019 08:00:00 AM,03/16/2019 06:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 65 St,NaN,NaN,"64,","22,"
303840,626079.0,NYC Saint Patrick's Day Parade,03/17/2022 11:00:00 AM,03/17/2022 04:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 65th St to 72nd St,NaN,NaN,"64,","22,"
303841,46217.0,NYC Saint Patrick's Day Parade,03/17/2012 11:00:00 AM,03/17/2012 05:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: 5th Ave Sidewalks-97th St to 90th St,NaN,NaN,"64,","22,"
303842,396022.0,NYC Saint Patrick's Day Parade,03/17/2018 08:00:00 AM,03/17/2018 06:57:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 65 St,NaN,NaN,"64,","22,"
303843,83824.0,NYC Saint Patrick's Day Parade,03/16/2013 08:00:00 AM,03/16/2013 08:00:00 PM,Parks Department,Special Event,Manhattan,"Central Park: Fifth Ave Sidewalks - 65th St to 72nd St ,Central Park: Fifth Ave Sidewalks - 79th St to 85th St ,Central Park: Fifth Ave Sidewalks - 60 St to 65 St",NaN,NaN,"64,","22,"
303844,200123.0,NYC Saint Patrick's Day Parade,03/17/2015 11:00:00 AM,03/17/2015 05:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 65 St,NaN,NaN,"64,","22,"
303845,332693.0,"NYC Saint Patrick's Day Parade, inc",03/17/2017 06:00:00 AM,03/17/2017 07:00:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 65 St,NaN,NaN,"64,","22,"


# Pride March

In [229]:
event_4 = selected_tuples[selected_tuples['Event Name'].str.contains ("NYC Pride March")]
event_4

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
109071,622217.0,2022 NYC Pride March - ADA Seating,06/26/2022 11:00:00 AM,06/26/2022 06:00:00 PM,Parks Department,Special Event,Manhattan,St. Vincent's Triangle: St. Vincent's Triangle,NaN,NaN,"2,","6,"
114278,478842.0,ADA Seating - 2019 NYC Pride March,06/30/2019 12:00:00 PM,06/30/2019 11:59:00 PM,Parks Department,Special Event,Manhattan,St. Vincent's Triangle: St. Vincent's Triangle,NaN,NaN,"2,","6,"
225821,195957.0,Heritage of Pride NYC Pride March 2015,06/28/2015 12:00:00 PM,06/28/2015 06:00:00 PM,Police Department,Parade,Manhattan,EAST 41 STREET between 5 AVENUE and PARK AV...,Full,Full Street Closure,"2, 5, 6,","13, 14, 17, 6,"
225822,260355.0,Heritage of Pride NYC Pride March 2016,06/26/2016 12:00:00 PM,06/26/2016 06:00:00 PM,Police Department,Parade,Manhattan,FIFTH AVENUE between EAST 41 STREET and WES...,Full,Full Street Closure,"2, 5, 6,","13, 14, 17, 6,"
225823,326438.0,Heritage of Pride NYC Pride March 2017,06/25/2017 12:00:00 PM,06/25/2017 07:00:00 PM,Police Department,Parade,Manhattan,EAST 41 STREET between FIFTH AVENUE and PAR...,Full,Full Street Closure,"2, 5, 6,","13, 14, 17, 6,"
303811,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
303812,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
303813,402516.0,NYC Pride March - ADA Seating Area,06/24/2018 12:00:00 PM,06/24/2018 06:00:00 PM,Parks Department,Special Event,Manhattan,St. Vincent's Triangle: NYC AIDS Memorial Park,NaN,NaN,"2,","6,"
303814,332370.0,NYC Pride March 2017 - Broadcast Area,06/25/2017 11:00:00 AM,06/25/2017 08:00:00 PM,Street Activity Permit Office,Street Event,Manhattan,WEST 25 STREET between 5 AVENUE and BROADWAY,South,Curb Lane Only,"5,","13,"
303815,652816.0,NYC Pride March Grandstand,06/26/2022 11:00:00 AM,06/26/2022 07:00:00 PM,Parks Department,Special Event,Manhattan,Worth Square: Worth Square,NaN,NaN,"5,","13,"


In [230]:
event_4 = event_4.drop(event_4[(event_4['Event Name'].str.contains ("Pride March")) & (event_4['Event Name'].str.contains ("ADA"))].index)
event_4 = event_4.drop(event_4[(event_4['Event Name'].str.contains ("Pride March")) & (event_4['Event Name'].str.contains ("Grandstand"))].index)
event_4 = event_4.drop(event_4[(event_4['Event Name'].str.contains ("Pride March")) & (event_4['Event Name'].str.contains ("Broadcast"))].index)
event_4

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
225821,195957.0,Heritage of Pride NYC Pride March 2015,06/28/2015 12:00:00 PM,06/28/2015 06:00:00 PM,Police Department,Parade,Manhattan,EAST 41 STREET between 5 AVENUE and PARK AV...,Full,Full Street Closure,"2, 5, 6,","13, 14, 17, 6,"
225822,260355.0,Heritage of Pride NYC Pride March 2016,06/26/2016 12:00:00 PM,06/26/2016 06:00:00 PM,Police Department,Parade,Manhattan,FIFTH AVENUE between EAST 41 STREET and WES...,Full,Full Street Closure,"2, 5, 6,","13, 14, 17, 6,"
225823,326438.0,Heritage of Pride NYC Pride March 2017,06/25/2017 12:00:00 PM,06/25/2017 07:00:00 PM,Police Department,Parade,Manhattan,EAST 41 STREET between FIFTH AVENUE and PAR...,Full,Full Street Closure,"2, 5, 6,","13, 14, 17, 6,"
303811,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
303812,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
437425,638616.0,The REAL NYC Pride March,06/26/2022 02:00:00 PM,06/26/2022 04:00:00 PM,Police Department,Parade,Manhattan,WEST 28 STREET between 9 AVENUE and 10 AVENU...,Full,Full Street Closure,"2, 4,","10, 6,"


# Times Square Ball

In [170]:
event_5 = selected_tuples[selected_tuples['Event Name'].str.contains ("New Year's Eve") & selected_tuples['Start Date/Time'].str.contains ("12/31") ]
event_5 = event_5.drop(event_5[(event_5['Event Name'].str.contains ("Cipriani"))].index)
event_5 = event_5.drop(event_5[(event_5['Event Name'].str.contains ("Party"))].index)
event_5

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
306267,43353.0,New Year's Eve,12/31/2012 06:00:00 PM,01/01/2013 12:15:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 42 STREET and WEST ...,Full,Full Street Closure,"4, 5,","14, 18,"
306277,242474.0,New Year's Eve,12/31/2015 12:00:00 AM,12/31/2015 11:59:00 PM,Parks Department,Special Event,Manhattan,Father Duffy Square: Father Duffy Square,NaN,NaN,"5,","18,"
306290,116615.0,New Year's Eve,12/31/2013 12:00:00 PM,12/31/2013 11:00:00 PM,Parks Department,Special Event,Manhattan,Father Duffy Square: Father Duffy Square,NaN,NaN,"5,","18,"
306298,92391.0,New Year's Eve - Street Closures,12/31/2013 01:00:00 PM,01/01/2014 05:00:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 42 STREET and WEST ...,Full,Full Street Closure,"5, 64,","14, 18, 22,"
306299,78947.0,New Year's Eve - Street Closures,12/31/2012 01:00:00 PM,01/01/2013 05:00:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 42 STREET and WEST ...,Full,Full Street Closure,"5, 64,","14, 18, 22,"
306300,247254.0,New Year's Eve - Street Closures,12/31/2015 06:00:00 PM,01/01/2016 12:00:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 43 STREET and WEST ...,Full,Full Street Closure,"5, 64,","14, 18, 22,"
306301,188756.0,New Year's Eve - Street Closures,12/31/2014 06:00:00 PM,01/01/2015 12:00:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 42 STREET and WEST ...,Full,Full Street Closure,"5, 64,","14, 18, 22,"
306302,317824.0,New Year's Eve - Street Closures,12/31/2016 12:00:00 AM,01/01/2017 05:00:00 PM,Street Activity Permit Office,Street Event,Manhattan,7 AVENUE between WEST 43 STREET and WEST ...,Full,Full Street Closure,"5, 64,","14, 18, 22,"
306303,42275.0,New Year's Eve - Street Closures,12/31/2011 01:00:00 PM,01/01/2012 05:00:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 42 STREET and WEST ...,Full,Full Street Closure,"5, 64,","14, 18, 22,"
306304,23536.0,New Year's Eve 2011-2012 (production permit),12/31/2011 06:00:00 PM,01/01/2012 12:15:00 AM,Street Activity Permit Office,Special Event,Manhattan,7 AVENUE between WEST 42 STREET and WEST ...,Both,Sidewalk and Curb Lane Closure,"4, 5,","14, 18,"


# Lunar New Year Parade

In [183]:
event_6 = selected_tuples[selected_tuples['Event Name'].str.contains ("Lunar New Year ") & selected_tuples['Event Location'].str.contains ("MOTT")]
event_6

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
106091,187838.0,16th NYC Lunar New Year Parade,02/22/2015 01:00:00 PM,02/22/2015 04:00:00 PM,Police Department,Parade,Manhattan,FORSYTH STREET between HESTER STREET and GRAN...,Full,Full Street Closure,"2, 3,","5,"
106398,255095.0,17th NYC Lunar New Year Parade,02/14/2016 01:00:00 PM,02/14/2016 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between GRAND STREET and CANAL ST...,Full,Full Street Closure,"2, 3,","5,"
106443,322570.0,18th Lunar New Year Parade,02/05/2017 01:00:00 PM,02/05/2017 04:00:00 PM,Police Department,Parade,Manhattan,HESTER STREET between BOWERY and MULBERRY STR...,Full,Full Street Closure,"2, 3,","5,"
106464,390232.0,19th Lunar New Year Parade,02/25/2018 01:00:00 PM,02/25/2018 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between GRAND STREET and CANAL ST...,Full,Full Street Closure,"2, 3,","5,"
109467,457965.0,20th Lunar New Year Parade,02/17/2019 01:00:00 PM,02/17/2019 04:00:00 PM,Police Department,Parade,Manhattan,FORSYTH STREET between GRAND STREET and DELAN...,Full,Full Street Closure,"2, 3,","5,"
257867,15075.0,Lunar New Year Parade,02/21/2010 01:00:00 PM,02/21/2010 03:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between HESTER STREET and BROOME ...,Both,Full Street Closure,"2, 3,","5, 7,"
257868,530067.0,Lunar New Year Parade,02/09/2020 01:00:00 PM,02/09/2020 04:00:00 PM,Police Department,Parade,Manhattan,FORSYTH STREET between GRAND STREET and RIVIN...,Full,Full Street Closure,"2, 3,","5,"
257869,3158.0,Lunar New Year Parade,02/01/2009 01:00:00 PM,02/01/2009 03:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between HESTER STREET and BROOME ...,Both,Full Street Closure,"2, 3,","5, 7,"
436014,614467.0,The 23rd Lunar New Year Parade,02/20/2022 01:00:00 PM,02/20/2022 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between BROOME STREET and CANAL ST...,Full,Full Street Closure,"2, 3,","5,"
461016,682208.0,the 25th Lunar New Year Parade,02/12/2023 01:00:00 PM,02/12/2023 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between BROOME STREET and CANAL ST...,Both,Full Street Closure,"2, 3,","5,"


# Three Kings Day Parade

In [186]:
event_7 = selected_tuples[selected_tuples['Event Name'].str.contains ("Annual Three Kings Day Parade")]
event_7

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
110008,184160.0,38th Annual Three Kings Day Parade,01/06/2015 11:00:00 AM,01/06/2015 01:00:00 PM,Police Department,Parade,Manhattan,EAST 106 STREET between 5 AVENUE and LEXINGT...,Both,Full Street Closure,"11,","23, 25,"
110016,251005.0,39th Annual Three Kings Day Parade,01/06/2016 11:00:00 AM,01/06/2016 01:00:00 PM,Police Department,Parade,Manhattan,3 AVENUE between EAST 106 STREET and EAST 1...,Both,Full Street Closure,"11,","23, 25,"
110125,311947.0,40th Annual Three Kings Day Parade,01/06/2017 11:00:00 AM,01/06/2017 01:00:00 PM,Police Department,Parade,Manhattan,EAST 106 STREET between PARK AVENUE and 3 AV...,Both,Full Street Closure,"11,","23, 25,"
110138,382319.0,41st Annual Three Kings Day Parade,01/05/2018 11:00:00 AM,01/05/2018 01:00:00 PM,Police Department,Parade,Manhattan,EAST 115 STREET between PARK AVENUE and MADI...,Both,Full Street Closure,"11,","23, 25,"
110147,447741.0,42nd Annual Three Kings Day Parade,01/04/2019 11:00:00 AM,01/04/2019 01:00:00 PM,Police Department,Parade,Manhattan,EAST 106 STREET between 5 AVENUE and PARK AV...,Both,Full Street Closure,"11,","23, 25,"
110156,523266.0,43rd Annual Three Kings Day Parade,01/06/2020 11:00:00 AM,01/06/2020 01:00:00 PM,Police Department,Parade,Manhattan,EAST 106 STREET between 5 AVENUE and PARK AV...,Both,Full Street Closure,"11,","23, 25,"
110170,681902.0,46th Annual Three Kings Day Parade,01/06/2023 11:00:00 AM,01/06/2023 01:00:00 PM,Police Department,Parade,Manhattan,EAST 106 STREET between 5 AVENUE and PARK AVE...,Both,Full Street Closure,"11,","23, 25,"


# Macy's 4th of July Fireworks

In [219]:
event_8 = selected_tuples[selected_tuples['Event Name'].str.contains ("Macy") & selected_tuples['Event Name'].str.contains ("Fireworks")]
event_8

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
106711,111094.0,2013 Macy's Fireworks,07/03/2013 11:59:00 PM,07/04/2013 11:59:00 PM,Street Activity Permit Office,Special Event,Manhattan,WEST 29 STREET between WEST SIDE HIGHWAY an...,South,Curb Lane Only,"4,","10,"
106822,150418.0,2014 Macy's Fireworks - Audiece viewing operation,07/04/2014 05:00:00 PM,07/04/2014 10:00:00 PM,Parks Department,Special Event,Manhattan,East River Waterfront Esplanade: Pier 15 Lower...,NaN,NaN,"1,","1,"
110153,481433.0,43rd Annual Macys Fireworks Show,07/04/2019 12:01:00 AM,07/04/2019 11:30:00 PM,Street Activity Permit Office,Production Event,Manhattan,JOHN STREET between WATER STREET and SOUTH ST...,North,Partial Sidewalk Closure,"1,","1,"
259284,61012.0,Macy's July 4 Fireworks Master Operations Loca...,07/04/2012 03:00:00 PM,07/04/2012 10:00:00 PM,Hudson River Park Trust,Special Event,Manhattan,"HRPT Piers: Pier 62 ,HRPT Piers: Pier 63",NaN,NaN,"4,","10,"
259309,587005.0,Macys Fireworks Marquee Strike,07/05/2021 05:30:00 AM,07/05/2021 09:00:00 AM,Street Activity Permit Office,Production Event,Manhattan,WEST 35 STREET between BROADWAY and 7 AVENUE,South,Curb Lane Only,"5,","14,"
302586,490114.0,NBC Macy's 4th of July Fireworks Parking,07/04/2019 12:00:00 AM,07/04/2019 11:59:00 PM,Parks Department,Special Event,Manhattan,East River Walk: East River Esplanade,NaN,NaN,"8,","19,"
302587,490114.0,NBC Macy's 4th of July Fireworks Parking,07/02/2019 12:00:00 AM,07/02/2019 11:59:00 PM,Parks Department,Special Event,Manhattan,East River Walk: East River Esplanade,NaN,NaN,"8,","19,"
302588,490114.0,NBC Macy's 4th of July Fireworks Parking,07/03/2019 12:00:00 AM,07/03/2019 11:59:00 PM,Parks Department,Special Event,Manhattan,East River Walk: East River Esplanade,NaN,NaN,"8,","19,"
302589,490114.0,NBC Macy's 4th of July Fireworks Parking,07/05/2019 12:00:00 AM,07/05/2019 11:59:00 PM,Parks Department,Special Event,Manhattan,East River Walk: East River Esplanade,NaN,NaN,"8,","19,"


In [220]:
event = "Macy's 4th of July Fireworks"
location = "East River Walk: East River Esplanade"
borough = "Manhattan"
no = 14
new_entries = pd.DataFrame({'Event Name': [event] * no,
                            'Start Date/Time':["07/04/2008 12:00:00 AM",
                                              "07/04/2009 12:00:00 AM",
                                              "07/04/2010 12:00:00 AM",
                                              "07/04/2011 12:00:00 AM",
                                              "07/04/2012 12:00:00 AM",
                                              "07/04/2013 12:00:00 AM",
                                              "07/04/2014 12:00:00 AM",
                                              "07/04/2015 12:00:00 AM",
                                              "07/04/2016 12:00:00 AM",
                                              "07/04/2017 12:00:00 AM",
                                              "07/04/2018 12:00:00 AM",
                                              "07/04/2019 12:00:00 AM",
                                              "07/04/2021 12:00:00 AM",
                                              "07/04/2022 12:00:00 AM",],
                            'End Date/Time': ["07/04/2008 11:59:00 PM",
                                             "07/04/2009 11:59:00 PM",
                                             "07/04/2010 11:59:00 PM",
                                             "07/04/2011 11:59:00 PM",
                                             "07/04/2012 11:59:00 PM",
                                             "07/04/2013 11:59:00 PM",
                                             "07/04/2014 11:59:00 PM",
                                             "07/04/2015 11:59:00 PM",
                                             "07/04/2016 11:59:00 PM",
                                             "07/04/2017 11:59:00 PM",
                                             "07/04/2018 11:59:00 PM",
                                             "07/04/2019 11:59:00 PM",
                                             "07/04/2021 11:59:00 PM",
                                             "07/04/2022 11:59:00 PM",],
                            'Event Borough': [borough] * no,
                            'Event Location': [location] * no})

In [221]:
event_8 = pd.concat([event_8, new_entries], ignore_index=True)
event_8 = event_8[event_8['Event Name']=="Macy's 4th of July Fireworks"]
event_8

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
9,NaN,Macy's 4th of July Fireworks,07/04/2008 12:00:00 AM,07/04/2008 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
10,NaN,Macy's 4th of July Fireworks,07/04/2009 12:00:00 AM,07/04/2009 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
11,NaN,Macy's 4th of July Fireworks,07/04/2010 12:00:00 AM,07/04/2010 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
12,NaN,Macy's 4th of July Fireworks,07/04/2011 12:00:00 AM,07/04/2011 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
13,NaN,Macy's 4th of July Fireworks,07/04/2012 12:00:00 AM,07/04/2012 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
14,NaN,Macy's 4th of July Fireworks,07/04/2013 12:00:00 AM,07/04/2013 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
15,NaN,Macy's 4th of July Fireworks,07/04/2014 12:00:00 AM,07/04/2014 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
16,NaN,Macy's 4th of July Fireworks,07/04/2015 12:00:00 AM,07/04/2015 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
17,NaN,Macy's 4th of July Fireworks,07/04/2016 12:00:00 AM,07/04/2016 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN
18,NaN,Macy's 4th of July Fireworks,07/04/2017 12:00:00 AM,07/04/2017 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN


In [231]:
list_event = pd.concat([event_1,
                       event_2,
                       event_3,
                       event_4,
                       event_5,
                       event_6,
                       event_7,
                       event_8], ignore_index=True)
list_event

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
0,11288.0,83rd Annual Macy's Thanksgiving Day Parade,11/26/2009 10:00:00 AM,11/26/2009 02:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,WEST 77 STREET between CENTRAL PARK WEST an...,Both,Full Street Closure,"4, 5, 7,","14, 20,"
1,75680.0,86th Annual Macy's Thanksgiving Day Parade,11/22/2012 09:00:00 AM,11/23/2012 12:30:00 AM,Street Activity Permit Office,Special Event,Manhattan,COLUMBUS AVENUE between WEST 77 STREET and ...,East,Curb Lane Only,"5, 64, 7,","14, 18, 20, 22,"
2,125155.0,87th Annual Macy's Thanksgiving Day Parade,11/28/2013 09:00:00 AM,11/28/2013 11:00:00 AM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 65...,NaN,NaN,"64,","22,"
3,126522.0,87th Annual Macy's Thanksgiving Day Parade,11/28/2013 09:00:00 AM,11/28/2013 12:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,COLUMBUS AVENUE between WEST 77 STREET and ...,East,Curb Lane Only,"5, 64, 7,","14, 18, 20, 22,"
4,186645.0,88th Annual Macy's Thanksgiving Day Parade,11/27/2014 09:00:00 AM,11/27/2014 12:00:00 PM,Street Activity Permit Office,Special Event,Manhattan,COLUMBUS AVENUE between WEST 77 STREET and ...,Full,Full Street Closure,"5, 64, 7,","14, 18, 20, 22,"
5,437785.0,92nd Annual Macy's Thanksgiving Day Parade,11/22/2018 12:01:00 AM,11/22/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 72...,NaN,NaN,"64,","22,"
6,506638.0,93rd Annual Macy's Thanksgiving Day Parade,11/28/2019 12:01:00 AM,11/28/2019 11:59:00 PM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 72...,NaN,NaN,"64,","22,"
7,667935.0,96th Annual Macy's Thanksgiving Day Parade,11/24/2022 12:00:00 AM,11/24/2022 11:59:00 PM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 65...,NaN,NaN,"64,","22,"
8,379815.0,Halloween Parade and Party,10/29/2017 12:50:00 PM,10/29/2017 01:20:00 PM,Police Department,Parade,Manhattan,GREENWICH STREET between NORTH MOORE STREET a...,Full,Full Street Closure,"1,","1,"
9,210271.0,Halloween Parade and Party,10/25/2015 01:00:00 PM,10/25/2015 01:30:00 PM,Police Department,Parade,Manhattan,GREENWICH STREET between NORTH MOORE STREET a...,Full,Full Street Closure,"1,","1,"


In [232]:
#Save the list of events dataset to a separate documents
list_event.to_csv('list_event.csv', index=False)

In [233]:
list_event_2018 = list_event[list_event['Start Date/Time'].str.contains ("2018")]
list_event_2018

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
5,437785.0,92nd Annual Macy's Thanksgiving Day Parade,11/22/2018 12:01:00 AM,11/22/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 72...,NaN,NaN,"64,","22,"
11,446834.0,Halloween Parade and Party,10/28/2018 12:50:00 PM,10/28/2018 01:20:00 PM,Police Department,Parade,Manhattan,GREENWICH STREET between NORTH MOORE STREET a...,Full,Full Street Closure,"1,","1,"
16,406904.0,Halloween Parade and Party,10/28/2018 01:00:00 PM,10/28/2018 03:00:00 PM,Parks Department,Special Event,Manhattan,Washington Market Park: Washington Market Park...,NaN,NaN,"1,","1,"
21,396022.0,NYC Saint Patrick's Day Parade,03/17/2018 08:00:00 AM,03/17/2018 06:57:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 6...,NaN,NaN,"64,","22,"
28,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
29,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
43,449845.0,New Year's Eve 2018-2019,12/31/2018 12:00:00 AM,12/31/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Father Duffy Square: Father Duffy Square,NaN,NaN,"5,","18,"
51,390232.0,19th Lunar New Year Parade,02/25/2018 01:00:00 PM,02/25/2018 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between GRAND STREET and CANAL ST...,Full,Full Street Closure,"2, 3,","5,"
61,382319.0,41st Annual Three Kings Day Parade,01/05/2018 11:00:00 AM,01/05/2018 01:00:00 PM,Police Department,Parade,Manhattan,EAST 115 STREET between PARK AVENUE and MADI...,Both,Full Street Closure,"11,","23, 25,"
75,NaN,Macy's 4th of July Fireworks,07/04/2018 12:00:00 AM,07/04/2018 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN


In [236]:
list_event_2018 = list_event_2018.drop(16)
list_event_2018

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
5,437785.0,92nd Annual Macy's Thanksgiving Day Parade,11/22/2018 12:01:00 AM,11/22/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 72...,NaN,NaN,"64,","22,"
11,446834.0,Halloween Parade and Party,10/28/2018 12:50:00 PM,10/28/2018 01:20:00 PM,Police Department,Parade,Manhattan,GREENWICH STREET between NORTH MOORE STREET a...,Full,Full Street Closure,"1,","1,"
21,396022.0,NYC Saint Patrick's Day Parade,03/17/2018 08:00:00 AM,03/17/2018 06:57:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 6...,NaN,NaN,"64,","22,"
29,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
43,449845.0,New Year's Eve 2018-2019,12/31/2018 12:00:00 AM,12/31/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Father Duffy Square: Father Duffy Square,NaN,NaN,"5,","18,"
51,390232.0,19th Lunar New Year Parade,02/25/2018 01:00:00 PM,02/25/2018 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between GRAND STREET and CANAL ST...,Full,Full Street Closure,"2, 3,","5,"
61,382319.0,41st Annual Three Kings Day Parade,01/05/2018 11:00:00 AM,01/05/2018 01:00:00 PM,Police Department,Parade,Manhattan,EAST 115 STREET between PARK AVENUE and MADI...,Both,Full Street Closure,"11,","23, 25,"
75,NaN,Macy's 4th of July Fireworks,07/04/2018 12:00:00 AM,07/04/2018 11:59:00 PM,NaN,NaN,Manhattan,East River Walk: East River Esplanade,NaN,NaN,NaN,NaN


In [238]:
#Save the list of events dataset to a separate documents
list_event_2018.to_csv('list_event_2018.csv', index=False)

In [244]:
check = pd.read_csv('list_event_2018.csv')
check

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Location Coordinates,Event Street Side,Street Closure Type,Community Board,Police Precinct
0,437785.0,92nd Annual Macy's Thanksgiving Day Parade,11/22/2018 12:01:00 AM,11/22/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Central Park: Central Park West Sidewalks - 72...,"[-77.430817, 37.386538]",NaN,NaN,"64,","22,"
1,446834.0,Halloween Parade and Party,10/28/2018 12:50:00 PM,10/28/2018 01:20:00 PM,Police Department,Parade,Manhattan,GREENWICH STREET between NORTH MOORE STREET a...,"[-74.010019, 40.720777]",Full,Full Street Closure,"1,","1,"
2,396022.0,NYC Saint Patrick's Day Parade,03/17/2018 08:00:00 AM,03/17/2018 06:57:00 PM,Parks Department,Special Event,Manhattan,Central Park: Fifth Ave Sidewalks - 60 St to 6...,"[-73.983532, 40.750144]",NaN,NaN,"64,","22,"
3,402554.0,NYC Pride March,06/24/2018 12:00:00 PM,06/24/2018 05:00:00 PM,Police Department,Parade,Manhattan,WEST 19 STREET between 7 AVENUE and 9 AVENU...,"[-74.005335, 40.718353]",Full,Full Street Closure,"2, 4, 5,","10, 13, 14, 6,"
4,449845.0,Times Square Ball,12/31/2018 12:00:00 AM,12/31/2018 11:59:00 PM,Parks Department,Special Event,Manhattan,Father Duffy Square: Father Duffy Square,"[-73.98718149999999, 40.760206]",NaN,NaN,"5,","18,"
5,390232.0,19th Lunar New Year Parade,02/25/2018 01:00:00 PM,02/25/2018 04:00:00 PM,Police Department,Parade,Manhattan,MOTT STREET between GRAND STREET and CANAL ST...,"[-73.996296, 40.719565]",Full,Full Street Closure,"2, 3,","5,"
6,382319.0,41st Annual Three Kings Day Parade,1/5/2018 11:00,1/5/2018 13:00,Police Department,Parade,Manhattan,EAST 115 STREET between PARK AVENUE and MADI...,"[-74.21326, 39.5975166666667]",Both,Full Street Closure,"11,","23, 25,"
7,NaN,Macy's 4th of July Fireworks,7/4/2018 0:00,7/4/2018 23:59,NaN,NaN,Manhattan,East River Walk: East River Esplanade,"[-73.99905186090359, 40.71599281291161]",NaN,NaN,NaN,NaN
